# CHESS DATA ANALYSIS

TEAM: Abraham Borg, Mehar Rekhi, Sarom Thin, Cristian Vazquez

INTRO: We want to analyze a data set of chess games and use the information to make predictions about who will win a game of chess. The data is taken from this source: https://www.kaggle.com/datasets/milesh1/35-million-chess-games?resource=download 

DESCRIPTION OF SOURCE DATA SET: The data shows all game moves for White and Black. The date of each game is specified. We are also given the results of each game. We know how many moves each player made. Finally, we know what the ELO rating of each player was at the time of the game. There is a lot of information that we are not interested in so that will need to be cleaned up. Also, the list of moves is a long string, but we want to have each move occupy its own dataframe column.

The complete dataset is 3.5 million games (filesize about 2GB), but for this experiment we initially selected the first 30,000 games for analysis. It was clear that this sample was heavily skewed for White. So instead we opted to randomly select 2 million games.   

The data set was loaded into the dataframe from the local machine, and then the pickle of the completely prepared dataset was loaded onto github.

PREDICTION AND FEATURES TO USE AS PREDICTORS: 
PREDICTORS:
- What were the most common first 8 moves for white and black each year? How have these opening moves changed over time?
- For each year in the data set, which pieces were most commonly left in play at the endgame? 
- Is there a piece in the game that is a good predictor of who will win? For example, if white loses a bishop first, does that correlate with more losses for white? 
- If a player still has their queen and the other player doesn't, does the player with the queen always win? 
- Can we build a model that predicts who will win a game based on the first 8 chess moves? 
- Can our model predict who will win based on the endgame pieces available to both players?

PREDICT: Who will win based on the openings for each player?

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
from matplotlib import rcParams
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

# allow output to span multiple output lines in the console
pd.set_option('display.max_columns', 500)

# switch to seaborn default stylistic parameters
# see the useful https://seaborn.pydata.org/tutorial/aesthetics.html

sns.set()
sns.set_context('paper') # 'talk' for slightly larger

# change default plot size
rcParams['figure.figsize'] = 9,7

In [2]:
# code in this cell from: 
# https://stackoverflow.com/questions/27934885/how-to-hide-code-from-cells-in-ipython-notebook-visualized-with-nbviewer
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to display/hide the code."></form>''')

# DATA PREPARATION

The data set is given with many columns that we don't need, data in the wrong format, and many records that are not useful. The data file is also very big. To make things easier we upload the data from the local machine and then generate a PKL file of the prepared dataframe.

In [3]:
filename = 'C:\\Users\\Abrah\\Desktop\\CST383 - Data Science\\Final Project\\Chess_Data.txt'
n = 3561470     #number of records in file
s = 300000      #desired sample size
skip = sorted(random.sample(range(n), n - s))

In [4]:
# column names, without the chess moves column
misc_columnNames = ['PNG_File_Pos - DELETE ME', 'Date', 'Game Result', 'W-ELO', 'B-ELO', 
                    'Num Moves', 'miscDate - DELETE ME', 'result - DELETE ME', 'wELO - DELETE ME', 'bELO - DELETE ME', 
                    'event date - DELETE ME', 'setup - DELETE ME', 'fen - DELETE ME', 'flag - DELETE ME', 'oyrange - DELETE ME', 
                    'bad len - DELETE ME']
delete_cols = ['PNG_File_Pos - DELETE ME', 'miscDate - DELETE ME', 'result - DELETE ME', 
               'wELO - DELETE ME', 'bELO - DELETE ME', 'event date - DELETE ME', 
               'setup - DELETE ME', 'fen - DELETE ME', 'flag - DELETE ME', 'oyrange - DELETE ME', 'bad len - DELETE ME']

In [5]:
# read all data except chess moves,
misc_chess_data = pd.read_csv(filename, comment = '#', infer_datetime_format = True, header = None, sep = ' ', on_bad_lines = 'skip', skiprows = skip)
misc_chess_data.drop(misc_chess_data.columns[16], axis = 1, inplace = True)
misc_chess_data.columns = misc_columnNames
misc_chess_data.drop(labels = ['PNG_File_Pos - DELETE ME', 'miscDate - DELETE ME', 'result - DELETE ME', 
               'wELO - DELETE ME', 'bELO - DELETE ME', 'event date - DELETE ME', 
               'setup - DELETE ME', 'fen - DELETE ME', 'flag - DELETE ME', 'oyrange - DELETE ME', 'bad len - DELETE ME'], axis = 1, inplace = True)


C:\Users\Abrah\AppData\Local\Temp\ipykernel_12772\3410824919.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  misc_chess_data = pd.read_csv(filename, comment = '#', infer_datetime_format = True, header = None, sep = ' ', on_bad_lines = 'skip', skiprows = skip)


In [6]:
misc_chess_data.shape

(300000, 5)

In [7]:
# sanity check
misc_chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves
0,2000.05.24,1/2-1/2,2851,2748,52
1,2000.06.23,1/2-1/2,2851,2748,75
2,2000.06.19,1-0,2851,None,73
3,2000.06.19,1-0,2851,None,71
4,2000.03.14,1-0,2851,None,93


In the above cell we can see that the B-ELO has None values. We will fix that shortly. Next we prepare isolate the chess moves from the other data to make it easier to prepare this portion of the data. The dataframe game_moves will be merged with the previous dataframe after processsing.

In [8]:
# Isolate game moves from everything else.
game_moves = pd.read_csv(filename, engine = 'python', sep = '###', on_bad_lines = 'skip', header = None, skiprows = skip)

# drop first column of game moves (this is the misc chess data)
game_moves.drop(game_moves.columns[0], axis = 1, inplace = True)

In [9]:
game_moves.shape

(300000, 1)

In [10]:
game_moves.head()

,1
0,W1.d4 B1.e6 W2.Nf3 B2.Nf6 W3.c4 B3.d5 W4.Nc3 ...
1,W1.d4 B1.e6 W2.c4 B2.b6 W3.a3 B3.Bb7 W4.Nc3 B...
2,W1.e4 B1.c5 W2.Nf3 B2.d6 W3.c3 B3.Nf6 W4.Be2 ...
3,W1.e4 B1.d5 W2.exd5 B2.Qxd5 W3.Nc3 B3.Qa5 W4....
4,W1.d4 B1.d6 W2.Nf3 B2.Nf6 W3.g3 B3.Nbd7 W4.Bg...


In [11]:
# split game moves df into columns for each move. 
game_moves = game_moves.iloc[:, 0].str.lstrip()
game_moves = game_moves.iloc[:].str.split(pat = ' ', expand = True)
game_moves.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326
0,W1.d4,B1.e6,W2.Nf3,B2.Nf6,W3.c4,B3.d5,W4.Nc3,B4.dxc4,W5.e4,B5.Bb4,W6.Bg5,B6.c5,W7.Bxc4,B7.cxd4,W8.Nxd4,B8.Qa5,W9.Bd2,B9.O-O,W10.Nc2,B10.Bxc3,W11.Bxc3,B11.Qg5,W12.Qe2,B12.Qxg2,W13.O-O-O,B13.Qxe4,W14.Rhg1,B14.g6,W15.Ne3,B15.e5,W16.f4,B16.Be6,W17.Bd3,B17.Qxf4,W18.Rgf1,B18.Qh4,W19.Be1,B19.Qa4,W20.Rxf6,B20.Nc6,W21.Rxe6,B21.Nd4,W22.Qg4,B22.Qxa2,W23.Bxg6,B23.hxg6,W24.Rxg6+,B24.fxg6,W25.Qxg6+,B25.Kh8,W26.Qh5+,B26.Kg8,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,W1.d4,B1.e6,W2.c4,B2.b6,W3.a3,B3.Bb7,W4.Nc3,B4.f5,W5.d5,B5.Nf6,W6.g3,B6.Na6,W7.Bg2,B7.Nc5,W8.Nh3,B8.Bd6,W9.O-O,B9.Be5,W10.Qc2,B10.O-O,W11.Rd1,B11.Qe7,W12.Be3,B12.Rab8,W13.Rac1,B13.Nce4,W14.Nxe4,B14.Nxe4,W15.Nf4,B15.c5,W16.dxc6,B16.Bxc6,W17.Nd3,B17.Bf6,W18.f3,B18.Nc5,W19.b4,B19.Nxd3,W20.Rxd3,B20.d5,W21.f4,B21.dxc4,W22.Qxc4,B22.Bxg2,W23.Kxg2,B23.Rf7,W24.b5,B24.Re8,W25.Rcd1,B25.e5,W26.Rd7,B26.Qe6,W27.Qxe6,B27.Rxe6,W28.Kf3,B28.exf4,W29.gxf4,B29.Rxd7,W30.Rxd7,B30.Re7,W31.Rxe7,B31.Bxe7,W32.a4,B32.Kf7,W33.Bd4,B33.Bd6,W34.e4,B34.g6,W35.h3,B35.Ke6,W36.Bc3,B36.Bc7,W37.Bb4,B37.Bd8,W38.e5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,

In [12]:
game_moves.dtypes

0      object
1      object
2      object
3      object
4      object
        ...  
322    object
323    object
324    object
325    object
326    object
Length: 327, dtype: object

In [13]:
# chess moves need to be strings, so convert objects to strings.
game_moves = game_moves.astype('string', copy=True, errors='raise')
game_moves.dtypes

0      string
1      string
2      string
3      string
4      string
        ...  
322    string
323    string
324    string
325    string
326    string
Length: 327, dtype: object

In [14]:
# merge misc data and game moves into one df.
chess_data = pd.concat([misc_chess_data, game_moves], axis = 1)

In [15]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326
0,2000.05.24,1/2-1/2,2851,2748,52,W1.d4,B1.e6,W2.Nf3,B2.Nf6,W3.c4,B3.d5,W4.Nc3,B4.dxc4,W5.e4,B5.Bb4,W6.Bg5,B6.c5,W7.Bxc4,B7.cxd4,W8.Nxd4,B8.Qa5,W9.Bd2,B9.O-O,W10.Nc2,B10.Bxc3,W11.Bxc3,B11.Qg5,W12.Qe2,B12.Qxg2,W13.O-O-O,B13.Qxe4,W14.Rhg1,B14.g6,W15.Ne3,B15.e5,W16.f4,B16.Be6,W17.Bd3,B17.Qxf4,W18.Rgf1,B18.Qh4,W19.Be1,B19.Qa4,W20.Rxf6,B20.Nc6,W21.Rxe6,B21.Nd4,W22.Qg4,B22.Qxa2,W23.Bxg6,B23.hxg6,W24.Rxg6+,B24.fxg6,W25.Qxg6+,B25.Kh8,W26.Qh5+,B26.Kg8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2000.06.23,1/2-1/2,2851,2748,75,W1.d4,B1.e6,W2.c4,B2.b6,W3.a3,B3.Bb7,W4.Nc3,B4.f5,W5.d5,B5.Nf6,W6.g3,B6.Na6,W7.Bg2,B7.Nc5,W8.Nh3,B8.Bd6,W9.O-O,B9.Be5,W10.Qc2,B10.O-O,W11.Rd1,B11.Qe7,W12.Be3,B12.Rab8,W13.Rac1,B13.Nce4,W14.Nxe4,B14.Nxe4,W15.Nf4,B15.c5,W16.dxc6,B16.Bxc6,W17.Nd3,B17.Bf6,W18.f3,B18.Nc5,W19.b4,B19.Nxd3,W20.Rxd3,B20.d5,W21.f4,B21.dxc4,W22.Qxc4,B22.Bxg2,W23.Kxg2,B23.Rf7,W24.b5,B24.Re8,W25.Rcd1,B25.e5,W26.Rd7,B26.Qe6,W27.Qxe6,B27.Rxe6,W28.Kf3,B28.exf4,W29.gxf4,B29.Rxd7,W30.Rxd7,B30.Re7,W31.Rxe7,B31.Bxe7,W32.a4,B32.Kf7,W33.Bd4,B33.Bd6,W34.e4,B34.g6,W35.h3,B35.Ke6,W36.Bc3,B36.Bc7,W37.Bb4,B37.Bd8,W38.e5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<N

The merging of the dataframes was successful. There are a few remaining steps for data preparation. We need to make sure we have the correct datatypes in place and also remove rows with missing data.

In [16]:
# remove rows with missing player ratings.
chess_data = chess_data[chess_data['B-ELO'] != 'None']
chess_data = chess_data[chess_data['W-ELO'] != 'None']

In [17]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326
0,2000.05.24,1/2-1/2,2851,2748,52,W1.d4,B1.e6,W2.Nf3,B2.Nf6,W3.c4,B3.d5,W4.Nc3,B4.dxc4,W5.e4,B5.Bb4,W6.Bg5,B6.c5,W7.Bxc4,B7.cxd4,W8.Nxd4,B8.Qa5,W9.Bd2,B9.O-O,W10.Nc2,B10.Bxc3,W11.Bxc3,B11.Qg5,W12.Qe2,B12.Qxg2,W13.O-O-O,B13.Qxe4,W14.Rhg1,B14.g6,W15.Ne3,B15.e5,W16.f4,B16.Be6,W17.Bd3,B17.Qxf4,W18.Rgf1,B18.Qh4,W19.Be1,B19.Qa4,W20.Rxf6,B20.Nc6,W21.Rxe6,B21.Nd4,W22.Qg4,B22.Qxa2,W23.Bxg6,B23.hxg6,W24.Rxg6+,B24.fxg6,W25.Qxg6+,B25.Kh8,W26.Qh5+,B26.Kg8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2000.06.23,1/2-1/2,2851,2748,75,W1.d4,B1.e6,W2.c4,B2.b6,W3.a3,B3.Bb7,W4.Nc3,B4.f5,W5.d5,B5.Nf6,W6.g3,B6.Na6,W7.Bg2,B7.Nc5,W8.Nh3,B8.Bd6,W9.O-O,B9.Be5,W10.Qc2,B10.O-O,W11.Rd1,B11.Qe7,W12.Be3,B12.Rab8,W13.Rac1,B13.Nce4,W14.Nxe4,B14.Nxe4,W15.Nf4,B15.c5,W16.dxc6,B16.Bxc6,W17.Nd3,B17.Bf6,W18.f3,B18.Nc5,W19.b4,B19.Nxd3,W20.Rxd3,B20.d5,W21.f4,B21.dxc4,W22.Qxc4,B22.Bxg2,W23.Kxg2,B23.Rf7,W24.b5,B24.Re8,W25.Rcd1,B25.e5,W26.Rd7,B26.Qe6,W27.Qxe6,B27.Rxe6,W28.Kf3,B28.exf4,W29.gxf4,B29.Rxd7,W30.Rxd7,B30.Re7,W31.Rxe7,B31.Bxe7,W32.a4,B32.Kf7,W33.Bd4,B33.Bd6,W34.e4,B34.g6,W35.h3,B35.Ke6,W36.Bc3,B36.Bc7,W37.Bb4,B37.Bd8,W38.e5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<N

In [18]:
chess_data.shape

(164793, 332)

In [19]:
# remove rows where the number of game moves is 0.
chess_data = chess_data[chess_data['Num Moves'] != 0]

In [20]:
chess_data.shape

(163878, 332)

In [21]:
# remove rows where the number of game moves is over 75 for each player (75 * 2 players)
chess_data = chess_data[chess_data['Num Moves'] <= 150]

In [22]:
# we only want games with openings that we can analyze, so get all games that 
# have num moves at least 16 (8 move each side)
chess_data = chess_data[chess_data['Num Moves'] >= 16]

In [23]:
chess_data.shape

(158431, 332)

In [24]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326
0,2000.05.24,1/2-1/2,2851,2748,52,W1.d4,B1.e6,W2.Nf3,B2.Nf6,W3.c4,B3.d5,W4.Nc3,B4.dxc4,W5.e4,B5.Bb4,W6.Bg5,B6.c5,W7.Bxc4,B7.cxd4,W8.Nxd4,B8.Qa5,W9.Bd2,B9.O-O,W10.Nc2,B10.Bxc3,W11.Bxc3,B11.Qg5,W12.Qe2,B12.Qxg2,W13.O-O-O,B13.Qxe4,W14.Rhg1,B14.g6,W15.Ne3,B15.e5,W16.f4,B16.Be6,W17.Bd3,B17.Qxf4,W18.Rgf1,B18.Qh4,W19.Be1,B19.Qa4,W20.Rxf6,B20.Nc6,W21.Rxe6,B21.Nd4,W22.Qg4,B22.Qxa2,W23.Bxg6,B23.hxg6,W24.Rxg6+,B24.fxg6,W25.Qxg6+,B25.Kh8,W26.Qh5+,B26.Kg8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2000.06.23,1/2-1/2,2851,2748,75,W1.d4,B1.e6,W2.c4,B2.b6,W3.a3,B3.Bb7,W4.Nc3,B4.f5,W5.d5,B5.Nf6,W6.g3,B6.Na6,W7.Bg2,B7.Nc5,W8.Nh3,B8.Bd6,W9.O-O,B9.Be5,W10.Qc2,B10.O-O,W11.Rd1,B11.Qe7,W12.Be3,B12.Rab8,W13.Rac1,B13.Nce4,W14.Nxe4,B14.Nxe4,W15.Nf4,B15.c5,W16.dxc6,B16.Bxc6,W17.Nd3,B17.Bf6,W18.f3,B18.Nc5,W19.b4,B19.Nxd3,W20.Rxd3,B20.d5,W21.f4,B21.dxc4,W22.Qxc4,B22.Bxg2,W23.Kxg2,B23.Rf7,W24.b5,B24.Re8,W25.Rcd1,B25.e5,W26.Rd7,B26.Qe6,W27.Qxe6,B27.Rxe6,W28.Kf3,B28.exf4,W29.gxf4,B29.Rxd7,W30.Rxd7,B30.Re7,W31.Rxe7,B31.Bxe7,W32.a4,B32.Kf7,W33.Bd4,B33.Bd6,W34.e4,B34.g6,W35.h3,B35.Ke6,W36.Bc3,B36.Bc7,W37.Bb4,B37.Bd8,W38.e5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<N

In [25]:
# we have too many columns because of all the NA values. Get dataframe up to col 155. That's 5 cols for misc data plus 150 cols
# for the chess moves (there will be a max of 150 moves)
chess_data = chess_data[chess_data.columns[:155]]

In [26]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
0,2000.05.24,1/2-1/2,2851,2748,52,W1.d4,B1.e6,W2.Nf3,B2.Nf6,W3.c4,B3.d5,W4.Nc3,B4.dxc4,W5.e4,B5.Bb4,W6.Bg5,B6.c5,W7.Bxc4,B7.cxd4,W8.Nxd4,B8.Qa5,W9.Bd2,B9.O-O,W10.Nc2,B10.Bxc3,W11.Bxc3,B11.Qg5,W12.Qe2,B12.Qxg2,W13.O-O-O,B13.Qxe4,W14.Rhg1,B14.g6,W15.Ne3,B15.e5,W16.f4,B16.Be6,W17.Bd3,B17.Qxf4,W18.Rgf1,B18.Qh4,W19.Be1,B19.Qa4,W20.Rxf6,B20.Nc6,W21.Rxe6,B21.Nd4,W22.Qg4,B22.Qxa2,W23.Bxg6,B23.hxg6,W24.Rxg6+,B24.fxg6,W25.Qxg6+,B25.Kh8,W26.Qh5+,B26.Kg8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2000.06.23,1/2-1/2,2851,2748,75,W1.d4,B1.e6,W2.c4,B2.b6,W3.a3,B3.Bb7,W4.Nc3,B4.f5,W5.d5,B5.Nf6,W6.g3,B6.Na6,W7.Bg2,B7.Nc5,W8.Nh3,B8.Bd6,W9.O-O,B9.Be5,W10.Qc2,B10.O-O,W11.Rd1,B11.Qe7,W12.Be3,B12.Rab8,W13.Rac1,B13.Nce4,W14.Nxe4,B14.Nxe4,W15.Nf4,B15.c5,W16.dxc6,B16.Bxc6,W17.Nd3,B17.Bf6,W18.f3,B18.Nc5,W19.b4,B19.Nxd3,W20.Rxd3,B20.d5,W21.f4,B21.dxc4,W22.Qxc4,B22.Bxg2,W23.Kxg2,B23.Rf7,W24.b5,B24.Re8,W25.Rcd1,B25.e5,W26.Rd7,B26.Qe6,W27.Qxe6,B27.Rxe6,W28.Kf3,B28.exf4,W29.gxf4,B29.Rxd7,W30.Rxd7,B30.Re7,W31.Rxe7,B31.Bxe7,W32.a4,B32.Kf7,W33.Bd4,B33.Bd6,W34.e4,B34.g6,W35.h3,B35.Ke6,W36.Bc3,B36.Bc7,W37.Bb4,B37.Bd8,W38.e5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,2000.05.21,1-0,2851,2637,51,W1.e4,B1.e5,W2.Nf3,B2.Nf6,W3.Nxe5,B3.d6,W4.Nf3,B4.Nxe4,W5.d4,B5.d5,W6.Bd3,B6.Be7,W7.O-O,B7.Nc6,W8.c4,B8.Nb4,W9.Be2,B9.O-O,W10.Nc3,B10.Bf5,W11.a3,B11.Nxc3,W12.bxc3,B12.Nc6,W13.Re1,B13.Bf6,W14.Bf4,B14.Ne7,W15.Qb3,B15.b6,W16.cxd5,B16.Nxd5,W17.Be5,B17.Bg4,W18.Rad1,B18.Be7,W19.h3,B19.Bh5,W20.g4,B20.Bg6,W21.Bg3,B21.Nf6,W22.Ne5,B22.Ne4,W23.Bf3,B23.Nxg3,W24.Nc6,B24.Qd6,W25.Nxe7+,B25.Kh8,W26.Bxa8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
10,2000.10.15,1/2-1/2,2849,2770,48,W1.c4,B1.c5,W2.Nf3,B2.Nf6,W3.g3,B3.d5,W4.cxd5,B4.Nxd5,W5.Bg2,B5.Nc6,W6.Nc3,B6.g6,W7.O-O,B7.Bg7,W8.Qa4,B8.Nb6,W9.Qb5,B9.Nd7,W10.d3,B10.O-O,W11.Be3,B11.Nd4,W12.Bxd4,B12.cxd4,W13.Ne4,B13.Qb6,W14.a4,B14.a6,W15.Qxb6,B15.Nxb6,W16.a5,B16.Nd5,W17.Nc5,B17.Rd8,W18.Nd2,B18.Rb8,W19.Nc4,B19.e6,W20.Rfc1,B20.Bh6,W21.Rcb1,B21.Bf8,W22.Nb3,B22.Bg7,W23.Bxd5,B23.Rxd5,W24.Nbd2,B24.e5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA

In [27]:
# we only care about the year that the game took place. reformat the date col to reflect that
# try a string function on the df.
new_column = chess_data['Date'].str.slice(0, 4, 1)
chess_data['Date'] = new_column

In [29]:
chess_data.shape

(158431, 155)

In [28]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
0,2000,1/2-1/2,2851,2748,52,W1.d4,B1.e6,W2.Nf3,B2.Nf6,W3.c4,B3.d5,W4.Nc3,B4.dxc4,W5.e4,B5.Bb4,W6.Bg5,B6.c5,W7.Bxc4,B7.cxd4,W8.Nxd4,B8.Qa5,W9.Bd2,B9.O-O,W10.Nc2,B10.Bxc3,W11.Bxc3,B11.Qg5,W12.Qe2,B12.Qxg2,W13.O-O-O,B13.Qxe4,W14.Rhg1,B14.g6,W15.Ne3,B15.e5,W16.f4,B16.Be6,W17.Bd3,B17.Qxf4,W18.Rgf1,B18.Qh4,W19.Be1,B19.Qa4,W20.Rxf6,B20.Nc6,W21.Rxe6,B21.Nd4,W22.Qg4,B22.Qxa2,W23.Bxg6,B23.hxg6,W24.Rxg6+,B24.fxg6,W25.Qxg6+,B25.Kh8,W26.Qh5+,B26.Kg8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2000,1/2-1/2,2851,2748,75,W1.d4,B1.e6,W2.c4,B2.b6,W3.a3,B3.Bb7,W4.Nc3,B4.f5,W5.d5,B5.Nf6,W6.g3,B6.Na6,W7.Bg2,B7.Nc5,W8.Nh3,B8.Bd6,W9.O-O,B9.Be5,W10.Qc2,B10.O-O,W11.Rd1,B11.Qe7,W12.Be3,B12.Rab8,W13.Rac1,B13.Nce4,W14.Nxe4,B14.Nxe4,W15.Nf4,B15.c5,W16.dxc6,B16.Bxc6,W17.Nd3,B17.Bf6,W18.f3,B18.Nc5,W19.b4,B19.Nxd3,W20.Rxd3,B20.d5,W21.f4,B21.dxc4,W22.Qxc4,B22.Bxg2,W23.Kxg2,B23.Rf7,W24.b5,B24.Re8,W25.Rcd1,B25.e5,W26.Rd7,B26.Qe6,W27.Qxe6,B27.Rxe6,W28.Kf3,B28.exf4,W29.gxf4,B29.Rxd7,W30.Rxd7,B30.Re7,W31.Rxe7,B31.Bxe7,W32.a4,B32.Kf7,W33.Bd4,B33.Bd6,W34.e4,B34.g6,W35.h3,B35.Ke6,W36.Bc3,B36.Bc7,W37.Bb4,B37.Bd8,W38.e5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,2000,1-0,2851,2637,51,W1.e4,B1.e5,W2.Nf3,B2.Nf6,W3.Nxe5,B3.d6,W4.Nf3,B4.Nxe4,W5.d4,B5.d5,W6.Bd3,B6.Be7,W7.O-O,B7.Nc6,W8.c4,B8.Nb4,W9.Be2,B9.O-O,W10.Nc3,B10.Bf5,W11.a3,B11.Nxc3,W12.bxc3,B12.Nc6,W13.Re1,B13.Bf6,W14.Bf4,B14.Ne7,W15.Qb3,B15.b6,W16.cxd5,B16.Nxd5,W17.Be5,B17.Bg4,W18.Rad1,B18.Be7,W19.h3,B19.Bh5,W20.g4,B20.Bg6,W21.Bg3,B21.Nf6,W22.Ne5,B22.Ne4,W23.Bf3,B23.Nxg3,W24.Nc6,B24.Qd6,W25.Nxe7+,B25.Kh8,W26.Bxa8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
10,2000,1/2-1/2,2849,2770,48,W1.c4,B1.c5,W2.Nf3,B2.Nf6,W3.g3,B3.d5,W4.cxd5,B4.Nxd5,W5.Bg2,B5.Nc6,W6.Nc3,B6.g6,W7.O-O,B7.Bg7,W8.Qa4,B8.Nb6,W9.Qb5,B9.Nd7,W10.d3,B10.O-O,W11.Be3,B11.Nd4,W12.Bxd4,B12.cxd4,W13.Ne4,B13.Qb6,W14.a4,B14.a6,W15.Qxb6,B15.Nxb6,W16.a5,B16.Nd5,W17.Nc5,B17.Rd8,W18.Nd2,B18.Rb8,W19.Nc4,B19.e6,W20.Rfc1,B20.Bh6,W21.Rcb1,B21.Bf8,W22.Nb3,B22.Bg7,W23.Bxd5,B23.Rxd5,W24.Nbd2,B24.e5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<N

In [30]:
# take out games without dates.
chess_data = chess_data[chess_data['Date'].str.isnumeric()]

In [31]:
chess_data.shape

(158431, 155)

In [32]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
0,2000,1/2-1/2,2851,2748,52,W1.d4,B1.e6,W2.Nf3,B2.Nf6,W3.c4,B3.d5,W4.Nc3,B4.dxc4,W5.e4,B5.Bb4,W6.Bg5,B6.c5,W7.Bxc4,B7.cxd4,W8.Nxd4,B8.Qa5,W9.Bd2,B9.O-O,W10.Nc2,B10.Bxc3,W11.Bxc3,B11.Qg5,W12.Qe2,B12.Qxg2,W13.O-O-O,B13.Qxe4,W14.Rhg1,B14.g6,W15.Ne3,B15.e5,W16.f4,B16.Be6,W17.Bd3,B17.Qxf4,W18.Rgf1,B18.Qh4,W19.Be1,B19.Qa4,W20.Rxf6,B20.Nc6,W21.Rxe6,B21.Nd4,W22.Qg4,B22.Qxa2,W23.Bxg6,B23.hxg6,W24.Rxg6+,B24.fxg6,W25.Qxg6+,B25.Kh8,W26.Qh5+,B26.Kg8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2000,1/2-1/2,2851,2748,75,W1.d4,B1.e6,W2.c4,B2.b6,W3.a3,B3.Bb7,W4.Nc3,B4.f5,W5.d5,B5.Nf6,W6.g3,B6.Na6,W7.Bg2,B7.Nc5,W8.Nh3,B8.Bd6,W9.O-O,B9.Be5,W10.Qc2,B10.O-O,W11.Rd1,B11.Qe7,W12.Be3,B12.Rab8,W13.Rac1,B13.Nce4,W14.Nxe4,B14.Nxe4,W15.Nf4,B15.c5,W16.dxc6,B16.Bxc6,W17.Nd3,B17.Bf6,W18.f3,B18.Nc5,W19.b4,B19.Nxd3,W20.Rxd3,B20.d5,W21.f4,B21.dxc4,W22.Qxc4,B22.Bxg2,W23.Kxg2,B23.Rf7,W24.b5,B24.Re8,W25.Rcd1,B25.e5,W26.Rd7,B26.Qe6,W27.Qxe6,B27.Rxe6,W28.Kf3,B28.exf4,W29.gxf4,B29.Rxd7,W30.Rxd7,B30.Re7,W31.Rxe7,B31.Bxe7,W32.a4,B32.Kf7,W33.Bd4,B33.Bd6,W34.e4,B34.g6,W35.h3,B35.Ke6,W36.Bc3,B36.Bc7,W37.Bb4,B37.Bd8,W38.e5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,2000,1-0,2851,2637,51,W1.e4,B1.e5,W2.Nf3,B2.Nf6,W3.Nxe5,B3.d6,W4.Nf3,B4.Nxe4,W5.d4,B5.d5,W6.Bd3,B6.Be7,W7.O-O,B7.Nc6,W8.c4,B8.Nb4,W9.Be2,B9.O-O,W10.Nc3,B10.Bf5,W11.a3,B11.Nxc3,W12.bxc3,B12.Nc6,W13.Re1,B13.Bf6,W14.Bf4,B14.Ne7,W15.Qb3,B15.b6,W16.cxd5,B16.Nxd5,W17.Be5,B17.Bg4,W18.Rad1,B18.Be7,W19.h3,B19.Bh5,W20.g4,B20.Bg6,W21.Bg3,B21.Nf6,W22.Ne5,B22.Ne4,W23.Bf3,B23.Nxg3,W24.Nc6,B24.Qd6,W25.Nxe7+,B25.Kh8,W26.Bxa8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
10,2000,1/2-1/2,2849,2770,48,W1.c4,B1.c5,W2.Nf3,B2.Nf6,W3.g3,B3.d5,W4.cxd5,B4.Nxd5,W5.Bg2,B5.Nc6,W6.Nc3,B6.g6,W7.O-O,B7.Bg7,W8.Qa4,B8.Nb6,W9.Qb5,B9.Nd7,W10.d3,B10.O-O,W11.Be3,B11.Nd4,W12.Bxd4,B12.cxd4,W13.Ne4,B13.Qb6,W14.a4,B14.a6,W15.Qxb6,B15.Nxb6,W16.a5,B16.Nd5,W17.Nc5,B17.Rd8,W18.Nd2,B18.Rb8,W19.Nc4,B19.e6,W20.Rfc1,B20.Bh6,W21.Rcb1,B21.Bf8,W22.Nb3,B22.Bg7,W23.Bxd5,B23.Rxd5,W24.Nbd2,B24.e5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<N

In [33]:
# we want the date to be a number.
chess_data = chess_data.astype({'Date': 'int'}, copy=True, errors='raise')

In [34]:
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
0,2000,1/2-1/2,2851,2748,52,W1.d4,B1.e6,W2.Nf3,B2.Nf6,W3.c4,B3.d5,W4.Nc3,B4.dxc4,W5.e4,B5.Bb4,W6.Bg5,B6.c5,W7.Bxc4,B7.cxd4,W8.Nxd4,B8.Qa5,W9.Bd2,B9.O-O,W10.Nc2,B10.Bxc3,W11.Bxc3,B11.Qg5,W12.Qe2,B12.Qxg2,W13.O-O-O,B13.Qxe4,W14.Rhg1,B14.g6,W15.Ne3,B15.e5,W16.f4,B16.Be6,W17.Bd3,B17.Qxf4,W18.Rgf1,B18.Qh4,W19.Be1,B19.Qa4,W20.Rxf6,B20.Nc6,W21.Rxe6,B21.Nd4,W22.Qg4,B22.Qxa2,W23.Bxg6,B23.hxg6,W24.Rxg6+,B24.fxg6,W25.Qxg6+,B25.Kh8,W26.Qh5+,B26.Kg8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2000,1/2-1/2,2851,2748,75,W1.d4,B1.e6,W2.c4,B2.b6,W3.a3,B3.Bb7,W4.Nc3,B4.f5,W5.d5,B5.Nf6,W6.g3,B6.Na6,W7.Bg2,B7.Nc5,W8.Nh3,B8.Bd6,W9.O-O,B9.Be5,W10.Qc2,B10.O-O,W11.Rd1,B11.Qe7,W12.Be3,B12.Rab8,W13.Rac1,B13.Nce4,W14.Nxe4,B14.Nxe4,W15.Nf4,B15.c5,W16.dxc6,B16.Bxc6,W17.Nd3,B17.Bf6,W18.f3,B18.Nc5,W19.b4,B19.Nxd3,W20.Rxd3,B20.d5,W21.f4,B21.dxc4,W22.Qxc4,B22.Bxg2,W23.Kxg2,B23.Rf7,W24.b5,B24.Re8,W25.Rcd1,B25.e5,W26.Rd7,B26.Qe6,W27.Qxe6,B27.Rxe6,W28.Kf3,B28.exf4,W29.gxf4,B29.Rxd7,W30.Rxd7,B30.Re7,W31.Rxe7,B31.Bxe7,W32.a4,B32.Kf7,W33.Bd4,B33.Bd6,W34.e4,B34.g6,W35.h3,B35.Ke6,W36.Bc3,B36.Bc7,W37.Bb4,B37.Bd8,W38.e5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,2000,1-0,2851,2637,51,W1.e4,B1.e5,W2.Nf3,B2.Nf6,W3.Nxe5,B3.d6,W4.Nf3,B4.Nxe4,W5.d4,B5.d5,W6.Bd3,B6.Be7,W7.O-O,B7.Nc6,W8.c4,B8.Nb4,W9.Be2,B9.O-O,W10.Nc3,B10.Bf5,W11.a3,B11.Nxc3,W12.bxc3,B12.Nc6,W13.Re1,B13.Bf6,W14.Bf4,B14.Ne7,W15.Qb3,B15.b6,W16.cxd5,B16.Nxd5,W17.Be5,B17.Bg4,W18.Rad1,B18.Be7,W19.h3,B19.Bh5,W20.g4,B20.Bg6,W21.Bg3,B21.Nf6,W22.Ne5,B22.Ne4,W23.Bf3,B23.Nxg3,W24.Nc6,B24.Qd6,W25.Nxe7+,B25.Kh8,W26.Bxa8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
10,2000,1/2-1/2,2849,2770,48,W1.c4,B1.c5,W2.Nf3,B2.Nf6,W3.g3,B3.d5,W4.cxd5,B4.Nxd5,W5.Bg2,B5.Nc6,W6.Nc3,B6.g6,W7.O-O,B7.Bg7,W8.Qa4,B8.Nb6,W9.Qb5,B9.Nd7,W10.d3,B10.O-O,W11.Be3,B11.Nd4,W12.Bxd4,B12.cxd4,W13.Ne4,B13.Qb6,W14.a4,B14.a6,W15.Qxb6,B15.Nxb6,W16.a5,B16.Nd5,W17.Nc5,B17.Rd8,W18.Nd2,B18.Rb8,W19.Nc4,B19.e6,W20.Rfc1,B20.Bh6,W21.Rcb1,B21.Bf8,W22.Nb3,B22.Bg7,W23.Bxd5,B23.Rxd5,W24.Nbd2,B24.e5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<N

In [35]:
chess_data.dtypes

Date            int32
Game Result    object
W-ELO          object
B-ELO          object
Num Moves       int64
                ...  
145            string
146            string
147            string
148            string
149            string
Length: 155, dtype: object

In [36]:
# convert Num Moves to 32 bit int to reduce df size
chess_data = chess_data.astype({'Num Moves': 'int'}, copy=True, errors='raise')

In [37]:
# the dtypes for B-ELO and W-ELO need to be the same. So we convert W-ELO and B-ELO to int
chess_data = chess_data.astype({'B-ELO': 'int'}, copy=True, errors='raise')
chess_data = chess_data.astype({'W-ELO': 'int'}, copy=True, errors='raise')

In [38]:
# change game result to string
chess_data = chess_data.astype({'Game Result': 'string'}, copy=True, errors='raise')

In [39]:
chess_data.dtypes

Date            int32
Game Result    string
W-ELO           int32
B-ELO           int32
Num Moves       int32
                ...  
145            string
146            string
147            string
148            string
149            string
Length: 155, dtype: object

In [40]:
# This code will be used to rename the col names.
def createMovesList1(x): # Number of moves is <= 9
  size = x.size
  return x.str.slice(3,size,1)
def createMovesList2(x): # Number of moves is <= 99
  size = x.size
  return x.str.slice(4,size,1)
def createMovesList3(x): # Number of moves is <= 100
  size = x.size
  return x.str.slice(5,size,1)

In [41]:
# using this code to rename the cols.
chess_data.iloc[:,5:23] = chess_data.iloc[:, 5:23].apply(createMovesList1)
chess_data.iloc[:,23:113] = chess_data.iloc[:, 23:113].apply(createMovesList2)
chess_data.iloc[:,113:] = chess_data.iloc[:, 113:].apply(createMovesList3)
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149
0,2000,1/2-1/2,2851,2748,52,d4,e6,Nf3,Nf6,c4,d5,Nc3,dxc4,e4,Bb4,Bg5,c5,Bxc4,cxd4,Nxd4,Qa5,Bd2,O-O,Nc2,Bxc3,Bxc3,Qg5,Qe2,Qxg2,O-O-O,Qxe4,Rhg1,g6,Ne3,e5,f4,Be6,Bd3,Qxf4,Rgf1,Qh4,Be1,Qa4,Rxf6,Nc6,Rxe6,Nd4,Qg4,Qxa2,Bxg6,hxg6,Rxg6+,fxg6,Qxg6+,Kh8,Qh5+,Kg8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2000,1/2-1/2,2851,2748,75,d4,e6,c4,b6,a3,Bb7,Nc3,f5,d5,Nf6,g3,Na6,Bg2,Nc5,Nh3,Bd6,O-O,Be5,Qc2,O-O,Rd1,Qe7,Be3,Rab8,Rac1,Nce4,Nxe4,Nxe4,Nf4,c5,dxc6,Bxc6,Nd3,Bf6,f3,Nc5,b4,Nxd3,Rxd3,d5,f4,dxc4,Qxc4,Bxg2,Kxg2,Rf7,b5,Re8,Rcd1,e5,Rd7,Qe6,Qxe6,Rxe6,Kf3,exf4,gxf4,Rxd7,Rxd7,Re7,Rxe7,Bxe7,a4,Kf7,Bd4,Bd6,e4,g6,h3,Ke6,Bc3,Bc7,Bb4,Bd8,e5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,2000,1-0,2851,2637,51,e4,e5,Nf3,Nf6,Nxe5,d6,Nf3,Nxe4,d4,d5,Bd3,Be7,O-O,Nc6,c4,Nb4,Be2,O-O,Nc3,Bf5,a3,Nxc3,bxc3,Nc6,Re1,Bf6,Bf4,Ne7,Qb3,b6,cxd5,Nxd5,Be5,Bg4,Rad1,Be7,h3,Bh5,g4,Bg6,Bg3,Nf6,Ne5,Ne4,Bf3,Nxg3,Nc6,Qd6,Nxe7+,Kh8,Bxa8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
10,2000,1/2-1/2,2849,2770,48,c4,c5,Nf3,Nf6,g3,d5,cxd5,Nxd5,Bg2,Nc6,Nc3,g6,O-O,Bg7,Qa4,Nb6,Qb5,Nd7,d3,O-O,Be3,Nd4,Bxd4,cxd4,Ne4,Qb6,a4,a6,Qxb6,Nxb6,a5,Nd5,Nc5,Rd8,Nd2,Rb8,Nc4,e6,Rfc1,Bh6,Rcb1,Bf8,Nb3,Bg7,Bxd5,Rxd5,Nbd2,e5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
11,2001,1-0,2849,2672,65,e4,e5,Nf3,Nc6,Bb5,a6,Ba4,Nf6,O-O,Be7,Re1,b5,Bb3,O-O,a4,Bb7,d3,d6,Nbd2,Re8,Nf1,h6,Bd2,Bf8,c4,bxc4,Bxc4,Rb8,Bc3,Ne7,Ng3,Ng6,d4,exd4,Qxd4,d5,exd5,Rxe1+,Rxe1,Nxd5,Rd1,Ngf4,Nf5,Qf6,Qxf6,gxf6,Bd4,Bc8,Ne3,Nxe3,fxe3,Ne6,Bxf6,Bg7,Bxg7,Kxg7,b3,Kf6,Rf1,Rb6,Nd4+,Kg7,Nf5+,Kh7,Ne7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<

In [42]:
# This function will be used to concatenate two strings from two arrays
def combineStrings(a1,a2):
  new_list = []
  for i in range(0, a1.size):
    new_list.append(a1[i] + a2[i])
  return new_list

In [43]:
# Creating a list of integer from 1 to 75
# Then covert list type from int to string
a = np.arange(1,76,1)
a_str = list(map(str,a))

In [44]:
# Create move piece names.
w_pieces = np.full(75, ['W'])
b_pieces = np.full(75, ['B'])
w_new_list = combineStrings(w_pieces, a_str)
b_new_list = combineStrings(b_pieces, a_str)

In [45]:
# Combine list but alternate between white and black piece.
values = [None] * (len(w_new_list) + len(b_new_list))
values[::2] = w_new_list
values[1::2] = b_new_list

In [46]:
# rename columns to show W1, B1, W2, B2, .....
keys = np.arange(0,150,1)
dictionary = dict(zip(keys,values))
chess_data.rename(columns = dictionary, inplace=True)
chess_data.head()

,Date,Game Result,W-ELO,B-ELO,Num Moves,W1,B1,W2,B2,W3,B3,W4,B4,W5,B5,W6,B6,W7,B7,W8,B8,W9,B9,W10,B10,W11,B11,W12,B12,W13,B13,W14,B14,W15,B15,W16,B16,W17,B17,W18,B18,W19,B19,W20,B20,W21,B21,W22,B22,W23,B23,W24,B24,W25,B25,W26,B26,W27,B27,W28,B28,W29,B29,W30,B30,W31,B31,W32,B32,W33,B33,W34,B34,W35,B35,W36,B36,W37,B37,W38,B38,W39,B39,W40,B40,W41,B41,W42,B42,W43,B43,W44,B44,W45,B45,W46,B46,W47,B47,W48,B48,W49,B49,W50,B50,W51,B51,W52,B52,W53,B53,W54,B54,W55,B55,W56,B56,W57,B57,W58,B58,W59,B59,W60,B60,W61,B61,W62,B62,W63,B63,W64,B64,W65,B65,W66,B66,W67,B67,W68,B68,W69,B69,W70,B70,W71,B71,W72,B72,W73,B73,W74,B74,W75,B75
0,2000,1/2-1/2,2851,2748,52,d4,e6,Nf3,Nf6,c4,d5,Nc3,dxc4,e4,Bb4,Bg5,c5,Bxc4,cxd4,Nxd4,Qa5,Bd2,O-O,Nc2,Bxc3,Bxc3,Qg5,Qe2,Qxg2,O-O-O,Qxe4,Rhg1,g6,Ne3,e5,f4,Be6,Bd3,Qxf4,Rgf1,Qh4,Be1,Qa4,Rxf6,Nc6,Rxe6,Nd4,Qg4,Qxa2,Bxg6,hxg6,Rxg6+,fxg6,Qxg6+,Kh8,Qh5+,Kg8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2000,1/2-1/2,2851,2748,75,d4,e6,c4,b6,a3,Bb7,Nc3,f5,d5,Nf6,g3,Na6,Bg2,Nc5,Nh3,Bd6,O-O,Be5,Qc2,O-O,Rd1,Qe7,Be3,Rab8,Rac1,Nce4,Nxe4,Nxe4,Nf4,c5,dxc6,Bxc6,Nd3,Bf6,f3,Nc5,b4,Nxd3,Rxd3,d5,f4,dxc4,Qxc4,Bxg2,Kxg2,Rf7,b5,Re8,Rcd1,e5,Rd7,Qe6,Qxe6,Rxe6,Kf3,exf4,gxf4,Rxd7,Rxd7,Re7,Rxe7,Bxe7,a4,Kf7,Bd4,Bd6,e4,g6,h3,Ke6,Bc3,Bc7,Bb4,Bd8,e5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,2000,1-0,2851,2637,51,e4,e5,Nf3,Nf6,Nxe5,d6,Nf3,Nxe4,d4,d5,Bd3,Be7,O-O,Nc6,c4,Nb4,Be2,O-O,Nc3,Bf5,a3,Nxc3,bxc3,Nc6,Re1,Bf6,Bf4,Ne7,Qb3,b6,cxd5,Nxd5,Be5,Bg4,Rad1,Be7,h3,Bh5,g4,Bg6,Bg3,Nf6,Ne5,Ne4,Bf3,Nxg3,Nc6,Qd6,Nxe7+,Kh8,Bxa8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
10,2000,1/2-1/2,2849,2770,48,c4,c5,Nf3,Nf6,g3,d5,cxd5,Nxd5,Bg2,Nc6,Nc3,g6,O-O,Bg7,Qa4,Nb6,Qb5,Nd7,d3,O-O,Be3,Nd4,Bxd4,cxd4,Ne4,Qb6,a4,a6,Qxb6,Nxb6,a5,Nd5,Nc5,Rd8,Nd2,Rb8,Nc4,e6,Rfc1,Bh6,Rcb1,Bf8,Nb3,Bg7,Bxd5,Rxd5,Nbd2,e5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
11,2001,1-0,2849,2672,65,e4,e5,Nf3,Nc6,Bb5,a6,Ba4,Nf6,O-O,Be7,Re1,b5,Bb3,O-O,a4,Bb7,d3,d6,Nbd2,Re8,Nf1,h6,Bd2,Bf8,c4,bxc4,Bxc4,Rb8,Bc3,Ne7,Ng3,Ng6,d4,exd4,Qxd4,d5,exd5,Rxe1+,Rxe1,Nxd5,Rd1,Ngf4,Nf5,Qf6,Qxf6,gxf6,Bd4,Bc8,Ne3,Nxe3,fxe3,Ne6,Bxf6,Bg7,Bxg7,Kxg7,b3,Kf6,Rf1,Rb6,Nd4+,Kg7,Nf5+,Kh7,Ne7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>

In [48]:
total_games = chess_data.values.size
def createGames(total_games):
  new_list = []
  for i in range(1, total_games):
    new_list.append('Game' + str(i))
  return new_list
index_list = createGames(total_games)

In [49]:
chess_data.index = index_list

ValueError: Length mismatch: Expected axis has 158431 elements, new values have 24556804 elements

We finally have a dataset that has been prepared and formatted to our needs.

In [50]:
chess_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158431 entries, 0 to 191261
Columns: 155 entries, Date to B75
dtypes: int32(4), string(151)
memory usage: 186.1 MB


In [51]:
chess_data.head(100)

,Date,Game Result,W-ELO,B-ELO,Num Moves,W1,B1,W2,B2,W3,B3,W4,B4,W5,B5,W6,B6,W7,B7,W8,B8,W9,B9,W10,B10,W11,B11,W12,B12,W13,B13,W14,B14,W15,B15,W16,B16,W17,B17,W18,B18,W19,B19,W20,B20,W21,B21,W22,B22,W23,B23,W24,B24,W25,B25,W26,B26,W27,B27,W28,B28,W29,B29,W30,B30,W31,B31,W32,B32,W33,B33,W34,B34,W35,B35,W36,B36,W37,B37,W38,B38,W39,B39,W40,B40,W41,B41,W42,B42,W43,B43,W44,B44,W45,B45,W46,B46,W47,B47,W48,B48,W49,B49,W50,B50,W51,B51,W52,B52,W53,B53,W54,B54,W55,B55,W56,B56,W57,B57,W58,B58,W59,B59,W60,B60,W61,B61,W62,B62,W63,B63,W64,B64,W65,B65,W66,B66,W67,B67,W68,B68,W69,B69,W70,B70,W71,B71,W72,B72,W73,B73,W74,B74,W75,B75
0,2000,1/2-1/2,2851,2748,52,d4,e6,Nf3,Nf6,c4,d5,Nc3,dxc4,e4,Bb4,Bg5,c5,Bxc4,cxd4,Nxd4,Qa5,Bd2,O-O,Nc2,Bxc3,Bxc3,Qg5,Qe2,Qxg2,O-O-O,Qxe4,Rhg1,g6,Ne3,e5,f4,Be6,Bd3,Qxf4,Rgf1,Qh4,Be1,Qa4,Rxf6,Nc6,Rxe6,Nd4,Qg4,Qxa2,Bxg6,hxg6,Rxg6+,fxg6,Qxg6+,Kh8,Qh5+,Kg8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,2000,1/2-1/2,2851,2748,75,d4,e6,c4,b6,a3,Bb7,Nc3,f5,d5,Nf6,g3,Na6,Bg2,Nc5,Nh3,Bd6,O-O,Be5,Qc2,O-O,Rd1,Qe7,Be3,Rab8,Rac1,Nce4,Nxe4,Nxe4,Nf4,c5,dxc6,Bxc6,Nd3,Bf6,f3,Nc5,b4,Nxd3,Rxd3,d5,f4,dxc4,Qxc4,Bxg2,Kxg2,Rf7,b5,Re8,Rcd1,e5,Rd7,Qe6,Qxe6,Rxe6,Kf3,exf4,gxf4,Rxd7,Rxd7,Re7,Rxe7,Bxe7,a4,Kf7,Bd4,Bd6,e4,g6,h3,Ke6,Bc3,Bc7,Bb4,Bd8,e5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,2000,1-0,2851,2637,51,e4,e5,Nf3,Nf6,Nxe5,d6,Nf3,Nxe4,d4,d5,Bd3,Be7,O-O,Nc6,c4,Nb4,Be2,O-O,Nc3,Bf5,a3,Nxc3,bxc3,Nc6,Re1,Bf6,Bf4,Ne7,Qb3,b6,cxd5,Nxd5,Be5,Bg4,Rad1,Be7,h3,Bh5,g4,Bg6,Bg3,Nf6,Ne5,Ne4,Bf3,Nxg3,Nc6,Qd6,Nxe7+,Kh8,Bxa8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
10,2000,1/2-1/2,2849,2770,48,c4,c5,Nf3,Nf6,g3,d5,cxd5,Nxd5,Bg2,Nc6,Nc3,g6,O-O,Bg7,Qa4,Nb6,Qb5,Nd7,d3,O-O,Be3,Nd4,Bxd4,cxd4,Ne4,Qb6,a4,a6,Qxb6,Nxb6,a5,Nd5,Nc5,Rd8,Nd2,Rb8,Nc4,e6,Rfc1,Bh6,Rcb1,Bf8,Nb3,Bg7,Bxd5,Rxd5,Nbd2,e5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
11,2001,1-0,2849,2672,65,e4,e5,Nf3,Nc6,Bb5,a6,Ba4,Nf6,O-O,Be7,Re1,b5,Bb3,O-O,a4,Bb7,d3,d6,Nbd2,Re8,Nf1,h6,Bd2,Bf8,c4,bxc4,Bxc4,Rb8,Bc3,Ne7,Ng3,Ng6,d4,exd4,Qxd4,d5,exd5,Rxe1+,Rxe1,Nxd5,Rd1,Ngf4,Nf5,Qf6,Qxf6,gxf6,Bd4,Bc8,Ne3,Nxe3,fxe3,Ne6,Bxf6,Bg7,Bxg7,Kxg7,b3,Kf6,Rf1,Rb6,Nd4+,Kg7,Nf5+,Kh7,Ne7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>

In [52]:
# we will upload pkl file to github and use it for visualization
chess_data.to_pickle("./chess_data.pkl", compression = 'zip')